In [98]:
import sqlite3

In [99]:
# Connect to the database
conn = sqlite3.connect('astronomia.db')

# Create a cursor object
cursor = conn.cursor()

print("Connected to the database successfully")

Connected to the database successfully


In [100]:
# elminate the tables if exist
cursor.execute('DROP TABLE IF EXISTS Observaciones')
cursor.execute('DROP TABLE IF EXISTS Telescopios')
cursor.execute('DROP TABLE IF EXISTS Estrellas')


### Create Tables, Estrellas (stars), Telescopios (Telescopes), Observaciones (Observations)

In [101]:
# Crear la tabla Estrellas
cursor.execute('''
CREATE TABLE Estrellas (
    ID INTEGER PRIMARY KEY,
    nombre TEXT,
    tipo_espectral TEXT,
    magnitud REAL
);
''')

# Crear la tabla Telescopios
cursor.execute('''
CREATE TABLE Telescopios (
    ID INTEGER PRIMARY KEY,
    nombre TEXT,
    apertura REAL
);
''')

# Crear la tabla Observaciones
cursor.execute('''
CREATE TABLE Observaciones (
    ID INTEGER PRIMARY KEY,
    ID_estrella INTEGER,
    hora_observacion TEXT,
    telescopio TEXT,
    FOREIGN KEY (ID_estrella) REFERENCES Estrellas(ID)
);
''')

### Insert Into Estrellas, Observaciones , Telescopes

In [102]:
# Insertar datos en la tabla Estrellas
cursor.execute('''
INSERT INTO Estrellas (ID, nombre, tipo_espectral, magnitud) VALUES
(1, 'Betelgeuse', 'M1', 0.42),
(2, 'Sirius', 'A1', -1.46),
(3, 'Vega', 'A0', 0.03),
(4, 'Proxima Centauri', 'M5', 11.05);
''')

# Insertar datos en la tabla Telescopios
cursor.execute('''
INSERT INTO Telescopios (ID, nombre, apertura) VALUES
(1, 'ALMA', 12.0),
(2, 'VLT', 8.2);
''')

# Insertar datos en la tabla Observaciones
cursor.execute('''
INSERT INTO Observaciones (ID, ID_estrella, hora_observacion, telescopio) VALUES
(1, 1, '2024-10-05 12:00', 'ALMA'),
(2, 2, '2024-10-06 14:30', 'VLT'),
(3, 3, '2024-10-07 16:00', 'ALMA'),
(4, 1, '2024-10-08 18:30', 'ALMA'),
(5, 4, '2024-10-09 20:45', 'VLT');
''')

# Confirmar los cambios y cerrar la conexión
conn.commit()
conn.close()

In [103]:
conn = sqlite3.connect("astronomia.db")
cursor = conn.cursor()


In [104]:
# Conectar a la base de datos
conn = sqlite3.connect('astronomia.db')
cursor = conn.cursor()

#nombre de las estrellas y magnitud
cursor.execute('''SELECT nombre, magnitud FROM Estrellas''')
estrellas = cursor.fetchall()
print('Estrellas: Nombre y Magnitud')
for estrella in estrellas:
    print(estrella)

# estrellas observadas y las horas de observacion
cursor.execute('''SELECT Estrellas.nombre, Observaciones.hora_observacion FROM Estrellas JOIN Observaciones ON Estrellas.ID = Observaciones.ID_estrella''')
observaciones = cursor.fetchall()
print('Estrellas observadas y horas de observacion:')
for observacion in observaciones:
    print(observacion)

#seleccionar todas las estrellas con magnitud mayor a 1
cursor.execute('''SELECT nombre FROM Estrellas WHERE magnitud > 1''')
estrellas_mayora1 = cursor.fetchall()
print('Estrellas con magnitud mayor a 1:')
for estrella in estrellas_mayora1:
    print(estrella)

#observaciones con telescopio alma
cursor.execute("SELECT COUNT(*) FROM Observaciones WHERE telescopio = 'ALMA'")
observaciones_alma = cursor.fetchone()[0]
print("Número de observaciones realizadas con el telescopio ALMA:", observaciones_alma)
# Cerrar la conexión
conn.close()

Estrellas: Nombre y Magnitud
('Betelgeuse', 0.42)
('Sirius', -1.46)
('Vega', 0.03)
('Proxima Centauri', 11.05)
Estrellas observadas y horas de observacion:
('Betelgeuse', '2024-10-05 12:00')
('Sirius', '2024-10-06 14:30')
('Vega', '2024-10-07 16:00')
('Betelgeuse', '2024-10-08 18:30')
('Proxima Centauri', '2024-10-09 20:45')
Estrellas con magnitud mayor a 1:
('Proxima Centauri',)
Número de observaciones realizadas con el telescopio ALMA: 3


In [106]:
conn = sqlite3.connect("astronomia.db")
cursor = conn.cursor()

cursor.execute('''
SELECT nombre 
FROM Estrellas 
WHERE ID IN (
    SELECT ID_estrella 
    FROM Observaciones 
    GROUP BY ID_estrella 
    HAVING COUNT(*) > 1
)
''')
estrellas_mas_de_una_vez = cursor.fetchall()
print("Estrellas observadas más de una vez:")
for estrella in estrellas_mas_de_una_vez:
    print(estrella)
    
cursor.execute('''
SELECT Estrellas.nombre, COUNT(Observaciones.ID) as num_observaciones
FROM Observaciones
JOIN Estrellas ON Observaciones.ID_estrella = Estrellas.ID
GROUP BY Estrellas.nombre
ORDER BY num_observaciones DESC
''')
estrellas_num_observaciones = cursor.fetchall()
print("\nNombre de las estrellas y número de observaciones, ordenadas por número de observaciones:")
for estrella in estrellas_num_observaciones:
    print(estrella)

cursor.execute('SELECT DISTINCT telescopio FROM Observaciones')
telescopios = cursor.fetchall()
print("\nTelescopios utilizados en las observaciones, sin duplicados:")
for telescopio in telescopios:
    print(telescopio)

cursor.execute('''
SELECT nombre 
FROM Estrellas 
WHERE ID IN (
    SELECT ID_estrella 
    FROM Observaciones 
    GROUP BY ID_estrella 
    HAVING COUNT(*) > (
        SELECT AVG(num_observaciones) 
        FROM (
            SELECT COUNT(*) as num_observaciones 
            FROM Observaciones 
            GROUP BY ID_estrella
        )
    )
)
''')
estrellas_mas_que_promedio = cursor.fetchall()
print("\nEstrellas observadas más veces que el promedio de observaciones por estrella:")
for estrella in estrellas_mas_que_promedio:
    print(estrella)

conn.close()

Estrellas observadas más de una vez:
('Betelgeuse',)

Nombre de las estrellas y número de observaciones, ordenadas por número de observaciones:
('Betelgeuse', 2)
('Vega', 1)
('Sirius', 1)
('Proxima Centauri', 1)

Telescopios utilizados en las observaciones, sin duplicados:
('ALMA',)
('VLT',)

Estrellas observadas más veces que el promedio de observaciones por estrella:
('Betelgeuse',)
